<a href="https://www.kaggle.com/code/ramsesmdlc/mnist-linear-classification?scriptVersionId=140225983" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# __0. Libraries__

In [ ]:
#Import Libraries 

import numpy as np # linear algebra
import pandas as pd # data processing

import matplotlib as mpl # data visualization
import matplotlib.pyplot as plt # data visualization ("pyplot module", a.k.a. "plt")

    #Each "pyplot" function makes some change to a figure: e.g., creates a figure, creates a...
    #...plotting area in a figure, plots some lines in a plotting area, decorates the plot with...
    #...labels, etc. The various plots we can utilize using Pyplot are Line Plot, Histogram, Scatter,...
    #...3D Plot, Image, Contour, and Polar.

    #Binary Classifier
from sklearn.linear_model import SGDClassifier # Machine Learning Model (Linear Classification)
from sklearn.model_selection import cross_val_predict #Generating predictions from the Training Set

    #Performance Measures used on Classification Models
from sklearn.metrics import classification_report #Classification Report (Confusion Matrix, Precision score, Recall score, and F1 Score)
from sklearn.metrics import confusion_matrix #Performance Measure (Confusion Matrix)
from sklearn.metrics import ConfusionMatrixDisplay #Performance Measure (Plot Confusion Matrix)

from sklearn.metrics import precision_score, recall_score #Performance Measure (Precision and Recall Score)
from sklearn.metrics import f1_score #Performance Measure (F1 Score)
from sklearn.metrics import precision_recall_curve #Performance Measure (Precision Recall Curve)
from sklearn.metrics import roc_curve #Performance Measure (ROC Curve)
from sklearn.metrics import roc_auc_score #Performance Measure (ROC Curve AUC Score)

    #Multiclass Classification
from sklearn.svm import SVC #Support Vector Classification (from Support Vector Machines)
from sklearn.multiclass import OneVsRestClassifier #One-vs-the-rest (OvR) multiclass classifier

# __1. Important__

Type of machine learning system to build:

1. Supervised Learning: __Linear Classification__
2. Batch Learning (also called "offline learning")
3. Model-based learning

Goal:
1. __It is to take an image of a handwritten single digit, and determine what that digit is.__

2. __Metric__: This competition is evaluated on the categorization accuracy of your predictions (the percentage of images you get correct).

# __2. Loading the Data - Training Dataset__

In [ ]:
training_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

## __2.1. General Information__

1. The data file contain __gray-scale images__ of hand-drawn digits, from 0 through 9.

2. Each image is __28 pixels in height and 28 pixels in width, for a total of 784 pixels in total.__

> __Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel__, with higher numbers meaning darker. 

> This pixel-value is an integer between 0 and 255, inclusive.

> __Each pixel is a feature__.

3. The __training data set, has 785 columns__. The first column, called "label", is the digit that was drawn by the user. The __rest of the columns contain the pixel-values of the associated image__.

4. __Each pixel column in the training set has a name like pixelx, where x is an integer between 0 and 783, inclusive__. 

# __3. Exploratory Data Analysis (EDA) - Training Dataset__

In [ ]:
def EDA(training_data):  
  # Print the "shape" of the dataframe
  print("\n" +'\033[1m','\033[94m',"Shape of the dataframe:",'\033[0m', training_data.shape, "\n")
    
  # Print the "keys" of the dataframe
  print("\n" +'\033[1m','\033[94m',"Keys of the dataframe:",'\033[0m', "\n")
  print(training_data.keys())  
    
  # Print the "head" of the dataframe
  print("\n" +'\033[1m','\033[94m',"Head of the dataframe:",'\033[0m', "\n")
  print(training_data.head())
  
  # Print the "general information" of the dataframe
  print("\n", '\033[1m','\033[94m',"Information of the dataframe:",'\033[0m', "\n")
  training_data.info()
 
  # Print the "number and percentage" of missing values per column" of the dataframe
  print("\n" +'\033[1m','\033[94m', "Number and percentage of missing values per column of the dataframe:", '\033[0m' + "\n")
  missing = training_data.isnull().sum()
  percent = missing / training_data.shape[0] * 100
  print(pd.concat([missing, percent], axis=1, keys=["Missing", "Percent"]))
    
# Call the function
EDA(training_data)

## __3.1. Adjusting the Training Dataset__

1. If we want to see the data (as an image), we should consider what is described in the section "2.1. General Information" (i.e. __the pixel's values of the image of every number is entirely displayed in one single row composed by 784 values); therefore, it is necessary rescaling the data (i.e. rescale each row of the dataframe from a shape of {1 x 784} pixels to {28 x 28} pixels.__

> For example: To locate this pixel ("pixel31") on the image of a specific number, "pixel31" indicates the pixel is in the fourth column from the left, and the second row from the top.

2. Before applying the "reshape" function, we need to transform our dataset from "Pandas dataframe" to a "Numpy Array", because the "Numpy Array" has the function "reshape" available.

In [ ]:
print("\n", '\033[1m','\033[94m',"Information (Shape and Type) of the Dataframe (Y and X variables separatelly):",'\033[0m', "\n")
#Separating the Y and X values (i.e. dependent and independent variables)
Y_training, X_training = training_data["label"],training_data.iloc[:, 1:785] 

#Printing the "shape" of the dependent and independent variables (separately)
print(Y_training.shape)
print(X_training.shape)

#Printing the "type" of the dependent and independent variables (separately)
print(type(Y_training))
print(type(X_training))

print("\n", '\033[1m','\033[94m',"Information (Shape) of the Numpy Array (Y and X variables separatelly):",'\033[0m', "\n")
#Transforming the "independent variables" (from Pandas Dataframe to Numpy Array)
array_X = X_training.to_numpy()
print(array_X)

#Transforming the "dependent variables" (from Pandas Dataframe to Numpy Array)
array_Y = Y_training.to_numpy()
print(array_Y)

#Printing the "shape" of the independent variables (as a Numpy Array)
print(array_X.shape) 

#Printing the "shape" of the independent variables (as a Numpy Array)
print(array_Y.shape) 

In [ ]:
#Printing the image of a number (just to check!)

#Digit of row N°5, counting from 0,1,2,3,4 (without rescaling)
    #If we see the section "3. Exploratory Data Analysis (EDA) - Training Dataset", the output should
    #...be the number zero (0)
digitx1 = array_X[4]
print("\n", '\033[1m','\033[94m',"Target Value of the Image:",'\033[0m', "\n")
print(Y_training[4])
print("\n", '\033[1m','\033[94m',"Value of every pixel of the image (28 x 28 pixels):",'\033[0m', "\n")
print(digitx1)

#Digit of row N°5 (with rescaling)
digitx1_image = digitx1.reshape(28, 28)

#The matplotlibe function "imshow":
    #Display data as an image.
    #The input may either be actual RGB(A) data, or 2D scalar data, which will be rendered as a...
    #...pseudocolor image. For displaying a grayscale image set up the colormapping using the...
    #...parameters cmap='gray', vmin=0, vmax=255.
    #The number of pixels used to render an image is set by the Axes size and the dpi of the figure.
print("\n", '\033[1m','\033[94m',"Image (28 x 28 pixels):",'\033[0m', "\n")
plt.imshow(digitx1_image,cmap="binary")
plt.axis("off")
plt.show 

In [ ]:
#Printing the image of a number (just to check!)

#Digit of row N°3, counting from 0,1,2 (without rescaling)
    #If we see the section "3. Exploratory Data Analysis (EDA) - Training Dataset", the output should
    #...be the number one (1)
digitx2 = array_X[2]
print("\n", '\033[1m','\033[94m',"Target Value of the Image:",'\033[0m', "\n")
print(Y_training[2])
print("\n", '\033[1m','\033[94m',"Value of every pixel of the image (28 x 28 pixels):",'\033[0m', "\n")
print(digitx2)

#Digit of row N°3 (with rescaling)
digitx2_image = digitx2.reshape(28, 28)

#The matplotlibe function "imshow":
    #Display data as an image.
    #The input may either be actual RGB(A) data, or 2D scalar data, which will be rendered as a...
    #...pseudocolor image. For displaying a grayscale image set up the colormapping using the...
    #...parameters cmap='gray', vmin=0, vmax=255.
    #The number of pixels used to render an image is set by the Axes size and the dpi of the figure.
print("\n", '\033[1m','\033[94m',"Image (28 x 28 pixels):",'\033[0m', "\n")
plt.imshow(digitx2_image,cmap="binary")
plt.axis("off")
plt.show 

# __4. Training Model (Binary Linear Classifier) - Training Dataset__

__Key ideas (Part 1):__

1. __Binary Classifier__: distinguish between two classes. Some models are:

> Logistic Regression

> Support Vector Machines

> SGD Classifier (i.e., __it is a "Linear Classifier"__ + __application of the optimization technique called "Stochastic Gradient Descent").__

2. __Multiclass Classifier__: distinguish between two or more classes. Some models are:

> SGD Classifier (i.e., __it is a "Linear Classifier"__ + __application of the optimization technique called "Stochastic Gradient Descent").__

> Random Forests Classifiers

> Naive Bayes Classifiers

__Key ideas (Part 2):__

***Linear Classifier:*** 

1. It is a type of machine learning __algorithm__ that can __classify data__ into different categories __based on a "linear function of the features"__.

> Extremely Important: __"linear function of the features, not the weights".__

> __Linear function of the features__: it means that the __output__ of the function is the __weighted sum of the input features__ plus a bias term. It does __not involve any nonlinear transformation of the features__ (cubic, quadratic, exponential,...). __It depends of the input data__. It is useful to __model relationships between the input and outputs__.

> __Linear function of the weights__: it means that the __output__ of the function is a __linear combination of the weights__. It does __not involve any nonlinear transformation of the weights. It depends of the model's parameter__. It is useful to __regularize or constrain the model complexity (using Norm L1, Norm L2, etc.)__

2. It can __learn a set of "weights" and "a threshold"__ that can separate the instances into different classes.

> __Threshold__: it is a value that __determines which class a data point belongs to based on its distance from the decision boundary__. It can be adjusted to optimize the performance of the classifier. 

3. It __can be represented by a line, a plane, or a hyperplane__ in the feature space, depending on the number of features. 

> __The "threshold" is not__ a __"line, a plane, or a hyperplane"__

> __The "decision boundary" is__ a __"line, a plane, or a hyperplane"__ that divides the input space into regions corresponding to different classes.

> The __decision boundary of a linear classifier__ is the __set of points that are equally distant from two or more classes.__

> __Hyperplane__: it is a generalization of a line, plane, etc. It is a __subspace of one dimension less than the ambient space__.
 
4. It can be trained using different methods: __Linear Discriminant Analysis (LDA)__, Naive Bayes, or __Stochastic Gradient Descent (SGD)__

5. __Because its linearity in the features,__ the advatanges and disavantages of a "Linear Classifier" are the followings:

> Advantages: fast, simple, and scalable to high-dimensional data.

> Disadvantages: they may not be able to capture complex nonlinear patterns in the data, and they may be sensitive to outliers and noise.

__Key ideas (Part 3):__

***Stochastic Gradient Descent:*** 

1. (__SGD__) Stochastic gradient descent and (__LDA__) linear discriminant analysis __are two different concepts__.

2. LDA: __it is a classification method__ that can be used to separate data into different classes based on a linear combination of features that best separates the classes by maximizing the ratio of between-class variance to within-class variance. 

3. SGD: __it is an optimization method__ that can be used to train various models, such as linear regression, logistic regression, support vector machines or neural networks.

> It does __not correspond__ to a specific family of machine learning models.

> It is an __iterative method that updates the parameters of a model__ by using the data to estimate the gradient of the cost function. 

> It can introduce noise and instability in the optimization process, and __may require careful tuning of the learning rate and other hyperparameters__, and it is __sensitive to feature scaling.__

4. We can __use SGD with__ two different types of machine learning models such as __"classifiers"__ and __"regressors"__.

> "Linear classifier" and a "Regressor" solve different kinds of problems.  

5. The main differences between a "Linear classifier" and a "Regressor" are the __type of output__ they produce, the __way they measure the error or loss of their predictions.

> The __"Linear classifier" model__ can __predict a discrete class label__ for a given input (for instance, output class labels such as positive or not negative, sushi or pizza, etc). It __uses metrics such as accuracy, precision, recall, or F1-score__ to evaluate its performance in the classification tasks.

> The __"Regressor Model"__ can __predict a continuous value__ for a given input, such as pressure and weight, etc. It outputs a numerical value, which can be more or less close to the true value, and uses __metrics such as mean squared error (MSE), root mean squared error (RMSE), or mean absolute error (MAE)__ to evaluate its performance. A regressor uses a function (such as a line, a curve, or a surface) to fit the data and estimate the output value for any given input.

In [ ]:
#Considering a "Binary Linear Classifier" approach, we are going to build a model to detect the number nine (9).

#Model to use: SGDClassifier. It is named in our code as "sgd_bin_clf"
    #SGDClassifier ("Linear Classifier" + optimization technique called "Stochastic Gradient Descent")

#Defining the two classes (because we are talking about a "Binary Classifier"):
    #Class N°1: all numbers that are nine (9) in the dataset of images.
    #Class N°2: all numbers that are not nine (9) in the dataset of images.

#Defining Class N°1: 
    #It works like a boolean (True or False), without deleting any row. Jut putting "True" the rows...
    #...with the "target value nine (9)".
y_train_9 = (Y_training == 9) 

#SGDClassifier 
    #This code implements a stochastic gradient descent learning routine which supports different...
    #...loss functions and penalties for classification.
    
    #It is important to shuffle the training data before fitting the model.
    
    #random_state
        #It is an argument from the function "SGDClassifier"
        #It is used for "shuffling the data", when "shuffle" is set to "True".
        #It is useful for reproducibility of outputs across multiple function calls.
        #If you dont want to change the global "seed" value and only want to set the "state" for one task,..
        #...random_state is used. 
    #random.seed()    
        #It is a function, not an argument.
        #It is little bit differente from the argument "random_state"
        #The effect of setting the "seed" is global as it will end up effecting all functions.

sgd_bin_clf = SGDClassifier(random_state = 42)
    #Both sets "X_training" and "y_train_9" have the same number of rows (i.e., 42000)
sgd_bin_clf.fit(X_training, y_train_9)

#The output of our model (based on the training set) apparently are good, considering that we should...
#...not detect the image of number nine (9) and the first three outputs fo the array show...
#..."False, False, False" because those number's images belong to the numbers "1,0,1" (accordingly...
#...to section "3. Exploratory Data Analysis (EDA) - Training Dataset")
print("\n", '\033[1m','\033[94m',"Output of the Training Phase:",'\033[0m', "\n")
sgd_bin_clf.predict(X_training)

# __5. Performance Measures - Training Model (Binary Classifier) - Training Dataset__

## __5.1. Confusion Matrix__

__Key ideas:__

1. The basic idea is to __count the number of time an item (instance) of a specific class (for example: Class W) is classified as another class (for example: Class H)__. In other words, this method allows us to extract information about the model's performance and __determine if the model is "confused" in discriminating between classes.__

2. This method can be __used for "Binary Classification" and "Multiclass Classification".__

3. Each __row__ of the "Confusion Matrix" __represents "True Class = Ground Truth".__

4. Each __column__ of the "Confusion Matrix" __represents "Predicted Class".__

5. In a ideal world, we want a Confusion Matrix that shows maximum metrics of "True Positive" and "True Negative" and minimum metrics of "False Positive" and "False Negative".

In [ ]:
#Step 1 (Cross-Validation)
    #It generate "cross-validated estimates" (also called "pseudo-predictions") for each input data point...
    #...In other words, this function returns for "each element in the input", the "prediction" that...
    #...was obtained for that "element when it was in the test set". during the "Cross-Validation process".
        #In this step we are going to generate "predictions" using the training data.        
        #Then those "predictions" will be compared to the known target values (because all data values..
        #...used in this Step 1 belong to the "training set").
    
    #cv:
        #The data is splitted according to the "cv" parameter. 
        #It is the "cross-validation generator" (default=None).
        #Determines the "cross-validation" splitting strategy. 
        #Possible inputs for "cv" are:
            #None (to use the default 5-fold cross validation).
            #int (to specify the number of folds in a ""(Stratified)KFold".
        #If the "estimator" (also called model, which in our case is called "sgd_bin_clf") is a...
        #..."classifier" and the "dependent variable" is either "binary or multiclass", the...
        #..."StratifiedKFold" is used. 

print("\n", '\033[1m','\033[94m',"Predictions from CV Process:",'\033[0m', "\n")
y_train_pred = cross_val_predict(sgd_bin_clf, X_training, y_train_9, cv=5)
print(y_train_pred)
    
#Step 2 (Confusion Matrix)
    #We are going to generate the "Confusion Matrix" using the "training data" and the "predictions"...
    #...(from the previous step; i.e., from the Step 1)
    
    #Confusion Matrix
        #Compute "confusion matrix" to evaluate the "accuracy" of a classification.
        #By definition a confusion matrix "C" is such that "Cij" is equal to the number of "observations...
        #...known" to be in group "i" (rows) and "predicted" to be in group "j" (columns)
        #In binary classification, the count is:
            #true negatives = C00 (left upper quadrant)
            #false negatives = C10 (left lower quadrant)
            #true positives = C11 (right lower quadrant)
            #false positives = C01 (right upper quadrant)

#The output of the confusion matrix will shows us that there are:
    # 36771 images classified correctly as non-9s
    # 1366  images classified wrongly as non-9s
    # 1041  images classified wrongly as 9s
    # 2822  images classified correclty as 9s
print("\n", '\033[1m','\033[94m',"Confusion Matrix:",'\033[0m', "\n")
cm = confusion_matrix(y_train_9, y_train_pred)
print(cm)

print("\n", '\033[1m','\033[94m',"Plot of the Confusion Matrix:",'\033[0m', "\n")
disp_confusion_matrix = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=sgd_bin_clf.classes_)
disp_confusion_matrix.plot()
plt.show()

## __5.2. Precision and Recall__

__Key ideas:__

1. __These metrics are considered more precise than the information from "Confusion Matrix"__ described on section "5.1. Confusion Matrix"; therefore, we are going to use them. By definition a confusion matrix "C" is such that "Cij" is equal to the number of "observations known" to be in group "i" (rows) and "predicted" to be in group "j" (columns):
 
 > true negatives = C00 (left upper quadrant)
 
 > false negatives = C10 (left lower quadrant)
 
 > true positives = C11 (right lower quadrant)
 
 > false positives = C01 (right upper quadrant)

2. __Precision__: this metric allow us to know how reliable is a model when it tries to classify "true positive" values. In other words, with this metric we are going to answer the following question: __What portion of all values classified as "positive" (i.e., "false positive" + "true positive") are really "true positive" values?__

> We should try to maximize this metric.

> This metric is __based on "detected values" ("True Positive" and "False Positive")__. In other words, this metric takes into account a group of values (correctly and wrongly) classified as positive. 

> __Precision Equation = TP / (TP+FP)__; where "TP: True Positive" and "FP: False Positive".

3. __Recall__: this metric allow us to know how many "positive values" were detected among "all possible positive values" (i.e., "detected positive" values and "non-detected postive" values).

> It is __also called "Sensitivity" or "True Positive Rate" (TPR).__

> "False positive" values, which are considered "negative samples" wrongly classified, are neglected by this metric. 

> This metric is __based on "detected values" and "non-detected postive" values.__

> __Recall Equation = TP / (TP+FN)__; where "TP: True Positive" and "FN: False Negative".

4. Relationship between Precision and Recall metrics

|Precision Score|Recall Score|Comment
|--|--|--|
|High|Low|The model correctly classify the "true positive" values, but due to the low value of the "Recall" metric (i.e., few points of data "detected as positive" values), we are computing the "Precision" metric with just a few points of data|
|Low|High|The model correctly detects the "positive values", but we need to be careful because the model can also detect too many "False Positive" values. Consequently, this impacts the "Precision" metric.|

In [ ]:
#Computation Precision and Recall

#Precision
    #The model will show (output) a Precision metric of 73.05% (i.e., the model will be correct...
    #...about a class assignment {in this case the number nine (9)} 73.05% of the time.)
print("\n", '\033[1m','\033[94m',"Precision Score:",'\033[0m', "\n")
print(precision_score(y_train_9, y_train_pred))

#Recall
    #The model will show (output) a Recall metric of 67.38% (i.e., the model will detect...
    #...67.38% of "positive values" {in this case the number nine (9)} among "all possible positive...
    #...values" (i.e., "detected positive" values and "non-detected postive" values). 
print("\n", '\033[1m','\033[94m',"Recall Score:",'\033[0m', "\n")
print(recall_score(y_train_9, y_train_pred))

## __5.3. F1 Score__

__Key ideas:__

1. This metric is very __useful when comparing models.__

2. It works like a __mean of the "Precision" and "Recall" metrics.__

3. It __ranges from 0 (low values of both "Precision" and "Recall" metric.) to +1 (high values of both "Precision" and "Recall" metric.)__ 

4. It __favours models with similar "Precision" and "Recall" scores__. It is important to take into account the following:

> In __some contexts it is better to have higher "Recall" scores__. For instance: in self-driving, detecting pedestrians with the cameras and sensors.

> In __some contexts it is better to have higher "Precision" scores__. For instance: detecting cancer or HIV on patients.

5. __F1 Equation = (2 x Precision x Recall)/(Precision + Recall)__

In [ ]:
#F1
    #The model will show (output) a F1 metric of 0.70.
print("\n", '\033[1m','\033[94m',"F1 Score:",'\033[0m', "\n")
print(f1_score(y_train_9, y_train_pred))

## __5.Extra. Classification Report (Confusion Matrix, Precision score, Recall score, and F1 score)__

***Extremely Important: with this "Classification Report" code we can save a lot of time. To be specific, we could avoid doing the sections***:

> 5.1. Confusion Matrix

> 5.2. Precision and Recall

> 5.3. F1 Score

1. __It will show us a summary of the most used performance metrics in classification__ algorithms. We will see:

> Confusion Matrix, Precision score, Recall score, F1 score

2. The __"Confusion Matrix values"__ are the same that we have seen in the section "5.1. Confusion Matrix" for "9s class", to be specific the values of: 

> Support "non-9s" class: (37812 = 36771 + 1041)

> Support "9s" class: (4188 = 1366 + 2822)

3. The output of the confusion matrix will shows us that there are:

|Values|Classification|
|--|--|
|36771 images classified correctly as non-9s| true negatives|
|1366  images classified wrongly as non-9s| false positives|
|1041  images classified wrongly as 9s| false negatives|
|2822  images classified correclty as 9s| true positives|

3. The __"Precision score"__ is the same that we have seen in the section "5.2. Precision and Recall" for "9s class", to be specific the value of 0.73

4. The __"Recall score"__ is the same that we have seen in the section "5.2. Precision and Recall" for "9s class", to be specific the value of 0.67

5. The __"F1 score"__ is the same that we have seen in the section "5.3. F1 Score" for "9s class", to be specific the value of 0.70

In [ ]:
#Classification Report
    #"Labels" of the "Classification Report"
class_labels = ["non-9s class","9s class"]

print("\n", '\033[1m','\033[94m',"Classification report:",'\033[0m', "\n")
cr = classification_report(y_train_9, y_train_pred,target_names=class_labels)
print(cr)

## __5.4. Precision and Recall Tradeoff (Plot of "Precision and Recall vs Decision Threshold")__

__Key ideas:__

1. This detailed analysis is useful if one considers deeply the relevance of the __"Precision and Recall Tradeoff" versus the "Decision Threshold"__. In some ways this analysis is based on the information described in section "5.3. F1 Score".

2. "Decision Threshold": it is a value that __determines which class a data point belongs to based on its distance from the decision boundary__; therefore, a change in the "Decision Threshold" also means a change in the composition of the quadrants that belong to the "Confusion Matrix", and a change in the "Precision", "Recall" and "F1" scores. The following table describes in a better way the __behavior of the scores ("Precision and Recall") when the "Decision Threshold" is modified__:

|Decision Threshold Value|Precision Score|Recall Score
|--|--|--|
|Increase|Increase (not always)|Decrease|
|Decrease|Decrease|Increase|

3. The __decision for a specific value for the "Decision Threshold" will depend on the context of our project__ (as was described in "5.3. F1 Score".)

4. __Important__: This graph will help if we are interested in __finding the "threshold value" from a specific "Precision Value" or "Recall Value".__

In [ ]:
#Finding the "scores" of our training dataset.

    #By default, the model "SGDClassifier" use a "Decision Threshold = 0".
    
    #Using the argument (method= "decision_function") of the function "cross_val_predict()" we are going...
    #...determine or predict the "score" (also called "confidence scores") of each instance of our...
    #...training set. 
        #These "scores" are going to be taken into account to make changes in the value...
        #...of the "Decision Threshold".
        #The score for a sample is proportional to the "signed distance" of that sample to the...
        #hyperplane (i.e. it is just a "distance")
            #The "signed distance" to the hyperplane (computed as the dot product between the...
            #...coefficients and the input sample, plus the intercept).
    
    #This method basically returns a Numpy array of scores, in which each score lies...
    #...to the right or left side of the "Hyperplane".

#The output show us positive and negative scores (confidence scores):
    #Positive scores: it means that the evaluated instance belongs to a the number nine (9) class.
    #Negative scores: it means that the evaluated instance belongs to a the non-number nine (9) class.
y_train_scores = cross_val_predict(sgd_bin_clf, X_training, y_train_9, cv=5, method="decision_function")
print("\n", '\033[1m','\033[94m',"Scores of the Training Set:",'\033[0m', "\n")
print(y_train_scores)
print("\n", '\033[1m','\033[94m',"Shape of the array of Scores of the Training Set:",'\033[0m', "\n")
print(y_train_scores.shape)

In [ ]:
#Finding the "Recall and Precision" metrics of the training set (i.e. compute precision-recall pairs for...
#...different thresholds.)

    #Extremely Important: This implementation is restricted to the "binary classification".
    
    #It is important to remember that in this step is computed the "Precision and Recall" based...
    #...on the "True binary labels" (i.e. the set "y_train_9") and "scores of the training set" (found...
    #...in the previous step).
        
    #In some way, this process is similar to section "5.2. Precision and Recall", in which we compute the...
    #..."Precision and Recall" metrics based on the "training data" and our "selected class" ("y_train_9")..
    #...But in this case we compute the "Precision and Recall" metrics based on the scores of ...
    #..."training data" and our "selected class" ("y_train_9").
    
    #Extremely Important: the "last precision and recall" values" are "1. and 0." respectively and do...
    #... not have a corresponding #threshold". This ensures that the graph starts on the y axis...
    #...Therefore, we are going to see the in the output that:
        #Shape of the array "precisions" = 42001 instea of 42000
        #Shape of the array "recalls" = 42001 instea of 42000

    #“thresholds”:
        #It is a variable that stores an array of values that are used to compute the...
        #..."Recall and Precision".
        
        #It contains the values of the "probability estimates" for the positive "class nine (9)"...
        #...that correspond to each point on the "Precision and Recall Tradeoff" analysis. For example:
            #The first element of the array "thresholds" is 1.8, which means that if the...
            #...classifier assigns a score of 1.8 or higher to an instance, it will predict that it...
            #...belongs to "class nine (9)". 
            
precisions, recalls, thresholds = precision_recall_curve(y_train_9, y_train_scores)
print("\n", '\033[1m','\033[94m',"Precision, Recall and Threshold",'\033[0m', "\n")
print(precisions, recalls, thresholds)
print("\n", '\033[1m','\033[94m',"Shape of the array precisions:",'\033[0m', "\n")
print(precisions.shape)
print("\n", '\033[1m','\033[94m',"Shape of the array recalls:",'\033[0m', "\n")
print(recalls.shape)
print("\n", '\033[1m','\033[94m',"Shape of the thresholds:",'\033[0m', "\n")
print(thresholds.shape)

In [ ]:
#Plotting the "Recall and Precision versus Decision Threshold" metrics of the training set...
#...(i.e. compute precision-recall pairs for different thresholds.)

    #Some portions of the graph will show (output) the "Precision Line" going down when the "threshold"...
    #...is raised. It could be strange, but sometimes happen.
    
    #Extremely Important: the "last precision and recall" values" are "1. and 0."" respectively and do...
    #... not have a corresponding #threshold". This ensures that the graph starts on the y axis...
    #...Therefore, we are going to see the in the output that:
        #Shape of the array "precisions" = 42001 instea of 42000
        #Shape of the array "recalls" = 42001 instea of 42000
        #precisions [:-1]
        #recalls [:-1]
    
    #We are using scores to plot this graph!
    
def plot_precision_recall_threshold (precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1],"b--",label="Precision")
    plt.plot(thresholds, recalls[:-1],"g-",label="Recall")
    plt.title("Recall and Precision versus Decision Threshold")
    plt.xlabel("Threshold")
    plt.legend(loc="center left")
    
plot_precision_recall_threshold (precisions, recalls, thresholds)
plt.show()

## __5.5. Precision and Recall Tradeoff (Plot of "Precision vs Recall")__

__Key ideas:__

1. The __decision for a specific value for the "Decision Threshold" will depend on the context of our project__ (as was described in "5.3. F1 Score".)

In [ ]:
#Plotting the "Recall vs Precision" metrics of the training set

    #We are going to take the values generated in section "5.4. Precision and Recall Tradeoff...
    #...(Plot of "Precision and Recall vs Decision Threshold")"
    
    #We are using scores to plot this graph!
plt.plot(recalls,precisions)
plt.title("Precision versus Recall")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.show()

## __5.6. ROC Curve__

__Key ideas:__

1. The ROC curve __shows how well or the quality of a binary classifier or multiclass classifiers can distinguish between two classes.__ It plots TPR vs FPR at different classification thresholds. 

> __Lowering the classification threshold classifies more items as positive, thus increasing both False Positives and True Positives__. 

2. ___Extremely Important___: Despite the fact that the __"ROC Curve" Analysis is similar to the "Precision and Recall Tradeoff" analysis, they are applied for different purposes__. The application is described in the following lines:

> Use"Precision and Recall Tradeoff" analysis: __if you care about "False Positives" or the "Positive Class" is rare.__

3. In the case of multiclass classification, TPR or FPR is obtained in the following ways:

> One-vs-Rest (OvR) scheme, that compares each class against all the others (assumed as one).

> One-vs-One (OvO) scheme, that compares every unique pairwise combination of classes.

4. ROC Curve (also called "Receiver Operating Characteristic" Curve) plots the "Recall", also called "Sensitivity" or "True Positive Rate" (TPR) against the "False Positive Rate" (FPR). 

> FPR: ratio of "negative values" wrongly classified as "positive values".

> TNR: ratio of "negative values" correctly classified as "negatve values". It is also called "Specificity".

> TNR = 1 - FPR

In [ ]:
#Finding the "False Positive Rate (FPR) and True Positive Rate (TPR)" metrics of the training set.

    #Extremely Important: This implementation is restricted to the "binary classification".
    
    #It is important to remember that in this step is computed the "False Positive Rate (FPR) and...
    #...True Positive Rate (TPR)" based on the "True binary labels" (i.e. the set "y_train_9")...
    #...and "scores of the training set" (found in the section "5.4. Precision and Recall...
    #...Tradeoff (Plot of "Precision and Recall vs Decision Threshold")".
    
    #The steps to find "False Positive Rate (FPR) and True Positive Rate (TPR)" metrics of the...
    #...training set are similar to those described on section "5.4. Precision and Recall...
    #...Tradeoff (Plot of "Precision and Recall vs Decision Threshold")"
    
    #“thresholds2”:
        #It is a variable that stores an array of values that are used to compute the...
        #...false positive rate (FPR) and the true positive rate (TPR) for the ROC curve.
        
        #It contains the values of the "probability estimates" for the positive "class nine (9)"...
        #...that correspond to each point on the ROC curve. For example:
            #The first element of the array "thresholds2" is 1.8, which means that if the...
            #...classifier assigns a score of 1.8 or higher to an instance, it will predict that it...
            #...belongs to "class nine (9)". 
        
fp_rate, tp_rate, thresholds2 = roc_curve(y_train_9, y_train_scores)

print("\n", '\033[1m','\033[94m',"False Positive Rate (FPR), True Positive Rate (TPR), Threshold",'\033[0m', "\n")
print(fp_rate, tp_rate, thresholds2)
print("\n", '\033[1m','\033[94m',"Shape of the array False Positive Rate (FPR):",'\033[0m', "\n")
print(fp_rate.shape)
print("\n", '\033[1m','\033[94m',"Shape of the array True Positive Rate (TPR):",'\033[0m', "\n")
print(tp_rate.shape)
print("\n", '\033[1m','\033[94m',"Shape of the thresholds:",'\033[0m', "\n")
print(thresholds2.shape)

In [ ]:
#Plotting the "ROC Curve"    
    #ROC curves typically feature "true positive rate" (TPR) on the Y axis, and...
    #..."false positive rate" (FPR) on the X axis.
    #The top left corner of the plot is the “ideal point" where FPR = 0, and TPR = 1. It means
        #In essence, a larger "area under the curve" (AUC) is usually a good sign that the model...
        #...has a good performance in the classifying task.
        #In essence, the "ROC Curve" away from the "Random Classfier" is usually a good sign that...
        #...the model has a good performance in the classifying task.
    #The “steepness” of ROC curves is also important, since it is ideal to maximize the TPR...
    #...while minimizing the FPR.   
    
def plot_fpr_tpr_threshold (fp_rate, tp_rate, label=None):
    plt.plot(fp_rate, tp_rate,"b-",linewidth = 1, label="ROC Curve")
    plt.plot([0,1], [0,1],"g--",linewidth = 3, label="Random Classifier")
    plt.title("ROC Curve (TPR vs FPR for all possible thresholds)")
    plt.xlabel("False Positive Rate (FPR)")
    plt.ylabel("True Positive Rate (TPR)")
    plt.legend(loc="lower right")
    
plot_fpr_tpr_threshold(fp_rate, tp_rate)
plt.show()

## __5.7. ROC AUC__

__Key ideas:__

1. This metric is very __useful when comparing models.__

> __Perfect classifier: AUC = 1__

> Random Classifier: AUC = 0.5 (as we see in the "diagonal line"of ROC Curve of the section "5.6. ROC Curve".

2. ROC Curve AUC (Area Under the Curve)

In [ ]:
#ROC AUC Score 
print("\n", '\033[1m','\033[94m',"ROC AUC Score:",'\033[0m', "\n")
print(roc_auc_score(y_train_9, y_train_scores))

# __6. Multiclass Classification__

## __6.1. Training Model (Multiclass Classifier) - Training Dataset__


__Key ideas (Part 1):__

1. __Binary Classifier__: distinguish between two classes; __however, there are several ways to use multiple "Binary Classifiers" to do the task of "Multiclass Classification"__. Some models are:

> Logistic Regression

> Support Vector Machines

> SGD Classifier (i.e., it is a "Linear Classifier" + application of the optimization technique called "Stochastic Gradient Descent").

2. __Multiclass Classifier__: distinguish between two or more classes. Some models are:

> SGD Classifier (i.e., it is a "Linear Classifier" + application of the optimization technique called "Stochastic Gradient Descent").

> Random Forests Classifiers

> Naive Bayes Classifiers

__Key ideas (Part 2):__

1. Some ways to use "Multiple Binary Classifiers" to do the task of "Multiclass Classification" are "One-vs-the-one (OvO) multiclass strategy" and "One-vs-the-rest (OvR) multiclass strategy"

2. One-vs-the-one (OvO) multiclass strategy: it is a multiclass classification technique that works by __training one classifier for each pair of classes in the data__.

> __Each classifier is trained on a subset of the data that contains only the samples from the two classes it is comparing__. 

> At prediction time (i.e. when we apply the "prediction" commands), __each classifier votes for one of the two classes it knows about__, and the class with the most votes is selected as the final prediction. 

> This technique can be __useful for algorithms that do not scale well with the number of samples__ (i.e. algoirthms that handling a lot of observations results in too much time or too many computational resources being necessary to be used), such as __kernel methods__.

> This strategy is usually slower than "One-vs-the-rest (OvR) multiclass strategy".

3. Kernel algorithms: they are a class of machine learning algorithms that use a __"kernel function (also called "kernel method")" to map the input data into a higher-dimensional feature space__, where linear methods can be applied __to solve nonlinear problems.__

> "Kernel algorithms" can be used for various tasks: classification, regression, clustering, dimensionality reduction, and anomaly detection.

> Examples of "kernel algorithms" are: support vector machines, kernel perceptron, kernel principal component analysis, kernel k-means, and Gaussian processes. 

4. __Kernel functions measure the similarity between two data points.__ 

> Examples of "kernel functions": linear,polynomial, radial basis function (RBF), sigmoid, and Laplacian. 

5. One-vs-the-rest (OvR) multiclass strategy: it is a multiclass classification technique that __consists in fitting one classifier per class. For each classifier, the class is fitted against all the other classes._-

> It is __computational efficient__ and easy to __interpret__ (since only "n" classes classifiers are needed).

> __This is the most commonly used strategy for multiclass classification and is a fair default choice__

In [ ]:
#One-vs-the-rest (OvR) multiclass strategy
    #Model is going to be named as "ovr_classifier"

    #We are going to see an explicit way to apply this model (fit the data and then make predictions)..
    #...(i.e., One-vs-the-rest (OvR) multiclass strategy) 
    #Although we have the option to use Binary Algorithms such as "Logistic Regression,...
    #...Support Vector Machines or SGD Classifier"
    
    #Extremely Important: I built 2 models (with different properties). The model that I am going
    #...to use for performance checking is called "Model N°1: ovr_classifier":
        #The data used for the "fitting process" in the following code are just the first 100 rows from...
        #..."the X training sets and Y training set" because it will take at least 30 min (or even more)...
        #...to use the entire "X and Y training set" (i.e., 42000 rows) for the "fitting process"...
        #...in my laptop.
        #The data used for the "predicting process" in the following code are rows in the range from...
        #...[101:42001] from the "X training sets".
    
    #We are going to use the "X training and Y training" Numpy arrays generated in the..
    #...section "3.1. Adjusting the Training Dataset".
    
    #SVC(): 
        #It means "Support Vector Classification"
        #It will work as an "estimator" argument in the function "OneVsRestClassifier()"
        
    #As a way to check that the "One-vs-the-rest (OvR) multiclass strategy" is working properly, I...
    #...use predict the value of the variable "digitx" (as I did in section...
    #..."3.1. Adjusting the Training Dataset").
    
#Model N°1: ovr_classifier"    
    #Important: "Performance Measures" (described in section 7. Performance Measures - Training Model...
    #...(Multiclass Classifier) - Training Dataset) are developed for this model; however this model...
    #...was not selected for the "Test Phase".
ovr_classifier = OneVsRestClassifier(SVC())
ovr_classifier.fit(array_X[0:100],array_Y[0:100])
ovr_classifier.predict(array_X[101:42001])

In [ ]:
#Model N°2: ovr_classifier1
    #Important: "Performance Measures" (described in section 7. Performance Measures - Training Model...
    #...(Multiclass Classifier) - Training Dataset) are not developed for this model; however this model...
    #...was selected for the "Test Phase".
ovr_classifier1 = OneVsRestClassifier(SVC())
ovr_classifier1.fit(array_X[0:20000],array_Y[0:20000])
ovr_classifier1.predict(array_X[20000:42001])

# __7. Performance Measures - Training Model (Multiclass Classifier) - Training Dataset__

## __7.1. Confusion Matrix (One-vs-the-rest {OvR} multiclass strategy)__

In [ ]:
#Step 1 (Cross-Validation in "One-vs-the-rest {OvR} multiclass strategy")
    #It generate "cross-validated estimates" (also called "pseudo-predictions") for each input data point...
    #...In other words, this function returns for "each element in the input", the "prediction" that...
    #...was obtained for that "element when it was in the test set". during the "Cross-Validation process".
        #In this step we are going to generate "predictions" using the training data.        
        #Then those "predictions" will be compared to the known target values (because all data values..
        #...used in this Step 1 belong to the "training set").
    
    #cv:
        #The data is splitted according to the "cv" parameter. 
        #It is the "cross-validation generator" (default=None).
        #Determines the "cross-validation" splitting strategy. 
        #Possible inputs for "cv" are:
            #None (to use the default 5-fold cross validation).
            #int (to specify the number of folds in a ""(Stratified)KFold".
        #If the "estimator" (also called model, which in our case is called "ovr_classifier") is a...
        #..."classifier" and the "dependent variable" is either "binary or multiclass", the...
        #..."StratifiedKFold" is used. 

print("\n", '\033[1m','\033[94m',"Predictions from CV Process in One-vs-the-rest {OvR} multiclass strategy:",'\033[0m', "\n")
y_train_pred_ovr = cross_val_predict(ovr_classifier, X_training[0:100], Y_training[0:100], cv=2)
print(y_train_pred_ovr)
    
#Step 2 (Confusion Matrix in "One-vs-the-rest {OvR} multiclass strategy")
    #We are going to generate the "Confusion Matrix" using the "training data" and the "predictions"...
    #...(from the previous step; i.e., from the Step 1)
    
    #Confusion Matrix
        #Compute "confusion matrix" to evaluate the "accuracy" of a classification.
        #The main diagonal shows us the "true positives" instances (i.e., correctly classified). Meanwhille..
        #...the rest of the values of the matrix show us "misclassified" instances...
        #...(i.e.,"false negatives" or "false positives").
        
#The output of the confusion matrix will shows us that there are:
    # 16 images correctly classified  as 1s
    # 9 images correctly classified  as 9s    
    # 3 images of 5s misclassified  as 3s
    # 2 images of 2s misclassified  as 6s
    
print("\n", '\033[1m','\033[94m',"Confusion Matrix in One-vs-the-rest {OvR} multiclass strategy:",'\033[0m', "\n")
cm_ovr = confusion_matrix(Y_training[0:100], y_train_pred_ovr[0:100])
print(cm_ovr)

print("\n", '\033[1m','\033[94m',"Plot of the Confusion Matrix in One-vs-the-rest {OvR} multiclass strategy:",'\033[0m', "\n")
disp_confusion_matrix_ovr = ConfusionMatrixDisplay(confusion_matrix=cm_ovr)
disp_confusion_matrix_ovr.plot()
plt.show()

## __7.2. Confusion Matrix - Image Representation (One-vs-the-rest {OvR} multiclass strategy)__

In [ ]:
#Confusion Matrix - Image Representation (One-vs-the-rest {OvR} multiclass strategy)
    #The main diagonal shows us the "true positives" instances (i.e., correctly classified).
    #The main diagonal shows us dark squares:
        #It could mean poor performance of the model in that specific class compared to other classes
        #It could mean few instances of that specific class
    #The main diagonal shows us brighter squares:
        #It could mean correctly classified data instances (good performance of the model...
        #...compared to other classes)
        
#The output of "Confusion Matrix - Image Representation" show us in the "Main diagonal" a good...
#...performance on "1s class" and apparently a regular performance in the other classes...
#...(considering obviously that we are using only 100 instances of the training dataset and not...
#...the entire training dataset).
plt.matshow(cm_ovr,cmap=plt.cm.gray)
plt.title("Confusion Matrix - Image Representation")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

## __7.3. Matrix of Error Rates - Image Representation (One-vs-the-rest {OvR} multiclass strategy)__

In [ ]:
#Error Analysis - Image Representation

    #This anaysis is derived from the technique...
    #..."Confusion Matrix"- Image Representation (One-vs-the-rest {OvR} multiclass strategy)

    #Step #1:Apply sum of the elements of an array (i.e., Numpy array) over a given axis.
        #Axis = 1 (i.e. all elements in every row = elements that runs horizontally across the columns).
    
        #keepdims (optional): bool
            #If this is set to "True", the axes which are reduced are left in the result as dimensions...
            #...with size one.
                #In this case the reduced axes is the "axis 1" (the columns); therefore the result will...
                #...show us only one column.           
print("\n", '\033[1m','\033[94m',"Sum of the elements of an array:",'\033[0m', "\n")                
row_sums_ovr = cm_ovr.sum(axis=1,keepdims=True)
print(row_sums_ovr)

    #Step #2: Finding the "error rates"
        #We are going to use the "misclassified instances" (i.e., all the instances that do not belong...
        #...to the main diagonal).
        #We are going to divide every "misclassified instance" of the...
        #..."Confusion Matrix - One-vs-the-rest {OvR} multiclass strategy" by the total number of...
        #...instances that belong to each class (based again in the info of the...
        #..."Confusion Matrix" - One-vs-the-rest {OvR} multiclass strategy.
            #Considering this approach, the shape of the matrix "cm_ovr" {output of section...
            #....7.1. Confusion Matrix (One-vs-the-rest {OvR} multiclass strategy} and the shape of...
            #...the array "row_sums_ovr" (output of Step #1) are very important (i.e. both have...
            #...the same quantity of rows at the  moment of the division).
print("\n", '\033[1m','\033[94m',"Matrix of Error Rates:",'\033[0m', "\n")                    
error_cm_ovr = cm_ovr / row_sums_ovr
print(error_cm_ovr)

    #Step #3: Plot on the errors
        #The "main diagonal" of the matrix "error_cm_ovr" (output of Step #2) is fill wih zeros...
        #...because the goal is to show the in the plot only the errors (i.e. the values that do not...
        #...belong to the "main diagonal" of the matrix "error_cm_ovr" {output of Step #2}).
        
#The output of the "Matrix of Error Rates - Image Representation" show us the following:
    #Bright square: From the point of view of "Predicted Label" (i.e., from a "column point of view"),...
    #...there are a lot misclassified images as "3s class". 
    #Bright square: From the point of view of "True Label" (i.e., from a "row point of view"), ...
    #...the model has troubles classifying true images of "5s" as "5s class". 
print("\n", '\033[1m','\033[94m',"Matrix of Error Rates - Image Representation:",'\033[0m', "\n")                    
np.fill_diagonal(error_cm_ovr,0)
plt.matshow(error_cm_ovr,cmap=plt.cm.gray)
plt.title("Matrix of Error Rates - Image Representation")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

## __7.4. Classification Report (Confusion Matrix, Precision score, Recall score, and F1 score) - One-vs-the-rest (OvR) multiclass strategy__

***Extremely Important: with this "Classification Report" code we can save a lot of time. To be specific, we could avoid doing the sections***:

> Confusion Matrix

> Precision and Recall

> F1 Score

1. __It will show us a summary of the most used performance metrics in classification__ algorithms. We will see:

> Confusion Matrix, Precision score, Recall score, F1 score

2. The __"Confusion Matrix values"__ are the same that we have seen in the section "6.1. Confusion Matrix (One-vs-the-rest {OvR} multiclass strategy)", to be specific the values of: 

> Support "3s class": (11 = 2+1+7+1)

> Support "9s class": (14 = 2+2+1+9)

In [ ]:
#Classification Report
    #"Labels" of the "Classification Report"
class_labels_ovr = ["0s class","1s class","2s class","3s class","4s class","5s class","6s class","7s class","8s class","9s class"]

from sklearn.metrics import  classification_report
print("\n", '\033[1m','\033[94m',"Classification report in One-vs-the-rest {OvR} multiclass strategy:",'\033[0m', "\n")
cr_ovr = classification_report(Y_training[0:100], y_train_pred_ovr[0:100],target_names=class_labels_ovr)
print(cr_ovr)

# __8. Evaluation of the Model - Test Dataset__

## __8.1. Adjusting the Test Dataset__

In [ ]:
#Dataframe of features of the test dataset
x_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
print("\n", '\033[1m','\033[94m',"Dataframe of test dataset",'\033[0m', "\n")
print(x_test)

print("\n", '\033[1m','\033[94m',"Information (Shape and Type) of the Dataframe (X variable):",'\033[0m', "\n")
#Printing the "shape" of the independent variable
print(x_test.shape)

#Printing the "type" of the independent variable
print(type(x_test))

print("\n", '\033[1m','\033[94m',"Information (Shape) of the Numpy Array (X variables):",'\033[0m', "\n")
#Transforming the "independent variable" (from Pandas Dataframe to Numpy Array)
array_x_test = x_test.to_numpy()
print(array_x_test)

#Printing the "shape" of the independent variable (as a Numpy Array)
print(array_x_test.shape) 

## __8.2. Applying Multiclass Classification {One-vs-the-rest (OvR) Multiclass strategy} - Test Dataset__

In [ ]:
#Applying the "model" learned from the training set.
    #This model came from section "6.1. Training Model (Multiclass Classifier) - Training Dataset"
    #This model is applied to the dataset that came from section "8.1. Adjusting the Test Dataset"
test_prediction = ovr_classifier1.predict(array_x_test)
print("\n", '\033[1m','\033[94m',"Array of the of predicted labels (test set)",'\033[0m', "\n")
test_prediction

## __8.3. Preparing the outputs to be submitted__

In [ ]:
#Converting the "NumPy array" in a "Dataframe"
    #The nump "NumPy array" came from section "8.2. Applying Multiclass Classification...
    #...{One-vs-the-rest (OvR) Multiclass strategy} - Test Dataset"
test_prediction_df = pd.DataFrame(test_prediction, columns = ['Label'])
print("\n", '\033[1m','\033[94m',"Dataframe of the of predicted labels (test set)",'\033[0m', "\n")
print(test_prediction_df)

In [ ]:
#Putting the" index" as a "column"
test_prediction_df['index'] = test_prediction_df.index
test_prediction_df

#Renaming the "columns"
test_prediction_df.rename(columns={'Label': 'Label', 'index': 'ImageId'}, inplace=True)
test_prediction_df

#Swapping the "columns"
columns_titles = ["ImageId","Label"]
test_prediction_df=test_prediction_df.reindex(columns=columns_titles)
test_prediction_df

In [ ]:
print("\n", '\033[1m','\033[94m',"Submmitted Dataframe",'\033[0m', "\n")
print(test_prediction_df)

In [ ]:
#Tranforming the file from "Dataframe" (previously created) to a "CSV File"
#Submission file format (according to Kaggle):  
    #Submit a csv file with exactly 28000 entries plus a header row. 
    #Your submission will show an error if you have extra columns (beyond ImageId and Label) or rows.
Submission_Final = test_prediction_df.to_csv('Submission_Final.csv', index = False)

# __References__

1. **Hands-on Machine Learning with Scikit Learn, Keras and TensorFlow (Second Edition)**
> Aurélien Gerón

## <span style="color:red"> __Opportunities for enhancement__</span>

1. Transformation of the data 

> Using Pipelines (Feature Scaling)

2. Apply "Data Augmentation" (shifting or adding noise to images)